In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
fed_rate = "./Resources/FEDFUNDS.csv"
fed_df = pd.read_csv(fed_rate)
fed_df.head()

,DATE,FEDFUNDS
0,7/1/1954,0.80
1,8/1/1954,1.22
2,9/1/1954,1.06
3,10/1/1954,0.85
4,11/1/1954,0.83


In [3]:
tenyr_rate = "./Resources/GS10.csv"
tenyr_df = pd.read_csv(tenyr_rate)
tenyr_df.head()

,DATE,GS10
0,1953-04-01,2.83
1,1953-05-01,3.05
2,1953-06-01,3.11
3,1953-07-01,2.93
4,1953-08-01,2.95


In [4]:
gdp_data = "./Resources/GDP_data.csv"
gdp_df = pd.read_csv(gdp_data)


In [5]:
gdp_df = gdp_df.rename(columns={"%age": "percent"})
gdp_df.head()

,DATE,GDP,Growth,percent,Recession
0,1/1/1947,243.164,NaN,NaN,NaN
1,4/1/1947,245.968,2.804,1.2%,0.0
2,7/1/1947,249.585,3.617,1.5%,0.0
3,10/1/1947,259.745,10.160,4.1%,0.0
4,1/1/1948,265.742,5.997,2.3%,0.0


In [6]:
gdp_df['DATE'] = pd.to_datetime(gdp_df['DATE'])
tenyr_df['DATE'] = pd.to_datetime(tenyr_df['DATE'])
fed_df['DATE'] = pd.to_datetime(fed_df['DATE'])

In [7]:
spread_df = pd.merge( fed_df, tenyr_df, how='outer', on=["DATE"])
spread_df['spread'] = spread_df['GS10'] - spread_df['FEDFUNDS']
spread_df.head(10)

,DATE,FEDFUNDS,GS10,spread
0,1954-07-01,0.80,2.30,1.50
1,1954-08-01,1.22,2.36,1.14
2,1954-09-01,1.06,2.38,1.32
3,1954-10-01,0.85,2.43,1.58
4,1954-11-01,0.83,2.48,1.65
5,1954-12-01,1.28,2.51,1.23
6,1955-01-01,1.39,2.61,1.22
7,1955-02-01,1.29,2.65,1.36
8,1955-03-01,1.35,2.68,1.33
9,1955-04-01,1.43,2.75,1.32


In [10]:
sql_df = pd.merge( spread_df, gdp_df, how='outer', on=["DATE"])
sql_df.head(10)

,DATE,FEDFUNDS,GS10,spread,GDP,Growth,percent,Recession
0,1954-07-01,0.80,2.30,1.50,390.996,4.875,1.3%,0.0
1,1954-08-01,1.22,2.36,1.14,NaN,NaN,NaN,NaN
2,1954-09-01,1.06,2.38,1.32,NaN,NaN,NaN,NaN
3,1954-10-01,0.85,2.43,1.58,399.734,8.738,2.2%,0.0
4,1954-11-01,0.83,2.48,1.65,NaN,NaN,NaN,NaN
5,1954-12-01,1.28,2.51,1.23,NaN,NaN,NaN,NaN
6,1955-01-01,1.39,2.61,1.22,413.073,13.339,3.3%,0.0
7,1955-02-01,1.29,2.65,1.36,NaN,NaN,NaN,NaN
8,1955-03-01,1.35,2.68,1.33,NaN,NaN,NaN,NaN
9,1955-04-01,1.43,2.75,1.32,421.532,8.459,2.0%,0.0


In [11]:
rds_connection_string = "postgres:samk4711@localhost:5432/ETL Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [14]:
engine.table_names()

['sql_rate']

In [13]:
sql_df.to_sql(name='sql_rate', con=engine, if_exists='append', index=False)